In [20]:
!pip install qiskit qiskit-aer

In [21]:
import torch
from qiskit import QuantumCircuit
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from collections import Counter
from qiskit.quantum_info import Statevector
import numpy as np

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [23]:
data_obj = load_breast_cancer()
data = torch.tensor(data_obj.data, dtype=torch.float32, device=device)  # Features
labels = torch.tensor(data_obj.target, dtype=torch.int64, device=device)  # Labels

In [24]:
data = (data - data.min(dim=0).values) / (data.max(dim=0).values - data.min(dim=0).values)

In [25]:
def encode_data_into_state(vector):
    """Encodes a vector into a quantum state using amplitude encoding."""
    norm = torch.norm(vector)
    vector = vector / norm  # Normalize
    num_qubits = int(np.ceil(np.log2(len(vector))))
    padded_vector = torch.cat(
        [vector, torch.zeros(2**num_qubits - len(vector), device=device)]
    )
    return padded_vector

In [30]:
def swap_test_distance(train_state, test_state):
    # Convert the PyTorch tensors to NumPy arrays (Qiskit works with NumPy arrays)
  train_state_np = train_state.numpy()
  test_state_np = test_state.numpy()

  # Convert the NumPy arrays into statevectors using Qiskit
  train_state_qiskit = Statevector(train_state_np)
  test_state_qiskit = Statevector(test_state_np)

  # Compute the inner product (overlap) between the two states
  inner_product = np.vdot(train_state_qiskit.data, test_state_qiskit.data)

  # Compute the overlap (absolute value squared of the inner product)
  overlap = np.abs(inner_product)

  # Compute the distance (1 - overlap)
  distance = 1 - overlap**2

  return 1 - overlap, distance

In [29]:
def tribrid_qknn_classifier(test_point, training_data, train_data_raw, labels, k, alpha=0.4, beta=0.3):
    """
    Tribrid kNN Classifier combining quantum fidelity, SWAP test overlap, and Euclidean distance.
    Args:
        test_point: Test sample (torch tensor).
        training_data: Quantum-encoded training data (list of tensors).
        train_data_raw: Raw training data (torch tensor).
        labels: Labels for the training data.
        k: Number of neighbors to consider.
        alpha: Weight for quantum fidelity (0 <= alpha <= 1).
        beta: Weight for SWAP test overlap (0 <= beta <= 1).
               (1 - alpha - beta) weight is applied to Euclidean distance.
    Returns:
        Predicted label for the test sample.
    """
    test_state = encode_data_into_state(test_point)
    distances = []

    # Compute tribrid distances
    for train_state, train_raw in zip(training_data, train_data_raw):
        # Quantum Fidelity Distance
        swap_overlap, quantum_fidelity_distance = swap_test_distance(test_state, train_state)

        # SWAP Test Overlap (direct overlap as a distance metric)
        #swap_overlap = 1 - torch.abs(torch.dot(test_state, train_state)).item()

        # Euclidean Distance
        euclidean_distance = torch.dist(test_point, train_raw).item()

        # Tribrid Distance
        tribrid_distance = (
            alpha * quantum_fidelity_distance +
            beta * swap_overlap +
            (1 - alpha - beta) * euclidean_distance
        )
        distances.append(tribrid_distance)

    # Find indices of the k nearest neighbors
    k_nearest_indices = np.argsort(distances)[:k]

    # Get the labels of the k nearest neighbors
    k_nearest_labels = labels[k_nearest_indices]
    most_common = Counter(k_nearest_labels.tolist()).most_common(1)
    return most_common[0][0]

In [31]:
train_data = data[:400]
test_data = data[400:]
train_labels = labels[:400]
test_labels = labels[400:]

# Encode training data into quantum states
training_states = [encode_data_into_state(vec) for vec in train_data]

# Predict labels for the test set
k_values = [3, 5, 7]
alphas = [0.3, 0.4, 0.5]  # Test different weightings for fidelity
betas = [0.2, 0.3, 0.4]   # Test different weightings for SWAP test overlap
best_accuracy = 0
best_k = None
best_alpha = None
best_beta = None

# Grid search over k, alpha, and beta
for alpha in alphas:
    for beta in betas:
        if alpha + beta >= 1.0:  # Ensure weights sum to <= 1
            continue
        for k in k_values:
            predicted_labels = []
            for test_point in test_data:
                predicted_label = tribrid_qknn_classifier(
                    test_point, training_states, train_data, train_labels, k, alpha, beta
                )
                predicted_labels.append(predicted_label)

            # Calculate accuracy
            predicted_labels = torch.tensor(predicted_labels, device=device)
            accuracy = accuracy_score(test_labels.cpu().numpy(), predicted_labels.cpu().numpy())
            print(f"alpha={alpha}, beta={beta}, k={k}, Accuracy: {accuracy * 100:.2f}%")

            # Update best parameters
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_k = k
                best_alpha = alpha
                best_beta = beta

print(f"Best alpha: {best_alpha}, Best beta: {best_beta}, Best k: {best_k}, Best Accuracy: {best_accuracy * 100:.2f}%")

# Final Evaluation with Best k, alpha, and beta
predicted_labels = []
for test_point in test_data:
    predicted_label = tribrid_qknn_classifier(
        test_point, training_states, train_data, train_labels, best_k, best_alpha, best_beta
    )
    predicted_labels.append(predicted_label)

predicted_labels = torch.tensor(predicted_labels, device=device)
final_accuracy = accuracy_score(test_labels.cpu().numpy(), predicted_labels.cpu().numpy())
print(f"Final Model Accuracy: {final_accuracy * 100:.2f}%")



alpha=0.3, beta=0.2, k=3, Accuracy: 96.45%
alpha=0.3, beta=0.2, k=5, Accuracy: 96.45%
alpha=0.3, beta=0.2, k=7, Accuracy: 96.45%
alpha=0.3, beta=0.3, k=3, Accuracy: 96.45%
alpha=0.3, beta=0.3, k=5, Accuracy: 97.04%
alpha=0.3, beta=0.3, k=7, Accuracy: 96.45%
alpha=0.3, beta=0.4, k=3, Accuracy: 95.27%
alpha=0.3, beta=0.4, k=5, Accuracy: 98.22%
alpha=0.3, beta=0.4, k=7, Accuracy: 96.45%
alpha=0.4, beta=0.2, k=3, Accuracy: 95.86%
alpha=0.4, beta=0.2, k=5, Accuracy: 97.04%
alpha=0.4, beta=0.2, k=7, Accuracy: 97.04%
alpha=0.4, beta=0.3, k=3, Accuracy: 95.27%
alpha=0.4, beta=0.3, k=5, Accuracy: 98.22%
alpha=0.4, beta=0.3, k=7, Accuracy: 96.45%
alpha=0.4, beta=0.4, k=3, Accuracy: 94.67%
alpha=0.4, beta=0.4, k=5, Accuracy: 98.22%
alpha=0.4, beta=0.4, k=7, Accuracy: 97.04%
alpha=0.5, beta=0.2, k=3, Accuracy: 95.27%
alpha=0.5, beta=0.2, k=5, Accuracy: 98.22%
alpha=0.5, beta=0.2, k=7, Accuracy: 96.45%
alpha=0.5, beta=0.3, k=3, Accuracy: 94.67%
alpha=0.5, beta=0.3, k=5, Accuracy: 97.63%
alpha=0.5, 